# <a>Fase 4<a>
Crearás una base de datos, definirás la estructura de las tablas necesarias, cargarás los datos proporcionados y realizarás consultas básicas y avanzadas para analizar la información, todo esto usando las tablas del proyecto de la empresa aliada.     
                                                                                                                               
1.Crear una nueva base de datos en SSMS para almacenar todas las tablas de datos                                               
proporcionadas.                                                                                                                 
                                                                                                                                
2.Definir la estructura de las tablas necesarias para los conjuntos de datos                                                 
(DIM_CATEGORY, DIM_PRODUCT, DIM_SEGMENT, DIM_CALENDAR, FACT_SALES).                                                             

### Se crean las tablas

In [ ]:
'''
CREATE TABLE DIM_CATEGORY (
    ID_CATEGORY INT PRIMARY KEY,
    CATEGORY NVARCHAR(255) NOT NULL
);

CREATE TABLE DIM_PRODUCT (
    MANUFACTURER NVARCHAR(255) NOT NULL,
    BRAND NVARCHAR(255) NOT NULL,
    ITEM NVARCHAR(255),
    ITEM_DESCRIPTION NVARCHAR(255) NOT NULL,
    CATEGORY INT NOT NULL, -- Relación con ID_CATEGORY en DIM_CATEGORY
    FORMAT NVARCHAR(255) NOT NULL,
    ATTR1 NVARCHAR(255),
    ATTR2 NVARCHAR(255) NOT NULL,
    ATTR3 NVARCHAR(255),
    FOREIGN KEY (CATEGORY) REFERENCES DIM_CATEGORY(ID_CATEGORY)
);

CREATE TABLE DIM_SEGMENT (
    CATEGORY INT NOT NULL, -- Relación con ID_CATEGORY en DIM_CATEGORY
    ATTR1 NVARCHAR(255) NOT NULL,
    ATTR2 NVARCHAR(255) NOT NULL,
    ATTR3 NVARCHAR(255),
    FORMAT NVARCHAR(255) NOT NULL,
    SEGMENT NVARCHAR(255) NOT NULL,
    FOREIGN KEY (CATEGORY) REFERENCES DIM_CATEGORY(ID_CATEGORY)
);

CREATE TABLE DIM_CALENDAR (
    WEEK NVARCHAR(50) NOT NULL,
    YEAR INT NOT NULL,
    MONTH INT NOT NULL,
    WEEK_NUMBER INT NOT NULL,
    DATE DATE NOT NULL
);

CREATE TABLE FACT_SALES (
    WEEK NVARCHAR(50) NOT NULL, -- Relación con WEEK en DIM_CALENDAR
    ITEM_CODE NVARCHAR(255) NOT NULL, -- Relación con ITEM en DIM_PRODUCT
    TOTAL_UNIT_SALES FLOAT NOT NULL,
    TOTAL_VALUE_SALES FLOAT NOT NULL,
    TOTAL_UNIT_AVG_WEEKLY_SALES FLOAT NOT NULL,
    REGION NVARCHAR(255) NOT NULL,
    FOREIGN KEY (WEEK) REFERENCES DIM_CALENDAR(WEEK),
    FOREIGN KEY (ITEM_CODE) REFERENCES DIM_PRODUCT(ITEM)
);

CREATE UNIQUE INDEX UX_DIM_CALENDAR_WEEK ON DIM_CALENDAR(WEEK);

CREATE UNIQUE INDEX UX_DIM_PRODUCT_ITEM ON DIM_PRODUCT(ITEM);
'''

### Se cargan los datos de los archivos .csv a las tablas en SQL Server

In [ ]:
'''
BULK INSERT DIM_CATEGORY
FROM 'C:\Users\Isaac\Desktop\IHD\EBAC DT\CIENCIA DE DATOS\PROYECTO RECKITT\DIM_CATEGORY.csv'
WITH (
    FIRSTROW = 2,              -- Ignora los encabezados
    FIELDTERMINATOR = ',',     -- Delimitador de campo
    ROWTERMINATOR = '\n',      -- Delimitador de fila
    CODEPAGE = '65001',        -- UTF-8 para caracteres especiales
    TABLOCK
);

BULK INSERT DIM_CALENDAR
FROM 'C:\Users\Isaac\Desktop\IHD\EBAC DT\CIENCIA DE DATOS\PROYECTO RECKITT\DIM_CALENDAR.csv'
WITH (
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '\n', 
    FIRSTROW = 2,
    CODEPAGE = '65001' -- UTF-8
);

-- Deshabilitar el índice único temporalmente
ALTER INDEX UX_DIM_PRODUCT_ITEM ON dbo.DIM_PRODUCT DISABLE;

BULK INSERT DIM_PRODUCT
FROM 'C:\Users\Isaac\Desktop\IHD\EBAC DT\CIENCIA DE DATOS\PROYECTO RECKITT\DIM_PRODUCT.csv'
WITH (
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '\n', 
    FIRSTROW = 2,
    CODEPAGE = '65001' -- UTF-8
);

-- Habilitar el índice nuevamente
ALTER INDEX UX_DIM_PRODUCT_ITEM ON dbo.DIM_PRODUCT REBUILD;


BULK INSERT DIM_SEGMENT
FROM 'C:\Users\Isaac\Desktop\IHD\EBAC DT\CIENCIA DE DATOS\PROYECTO RECKITT\DIM_SEGMENT.csv'
WITH (
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '\n', 
    FIRSTROW = 2,
    CODEPAGE = '65001' -- UTF-8
);


BULK INSERT FACT_SALES
FROM 'C:\Users\Isaac\Desktop\IHD\EBAC DT\CIENCIA DE DATOS\PROYECTO RECKITT\FACT_SALES.csv'
WITH (
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '\n', 
    FIRSTROW = 2,
    LASTROW = 10,
    CODEPAGE = '65001'
);
'''

### Consultas Básicas

In [ ]:
'''
Verificar la cantidad de registros por tabla
SELECT COUNT(*) AS Total_Registros FROM DIM_CATEGORY;
SELECT COUNT(*) AS Total_Registros FROM DIM_PRODUCT;
SELECT COUNT(*) AS Total_Registros FROM DIM_SEGMENT;
SELECT COUNT(*) AS Total_Registros FROM DIM_CALENDAR;
SELECT COUNT(*) AS Total_Registros FROM FACT_SALES;

Verificar las primeras filas de cada tabla
SELECT TOP 10 * FROM DIM_CATEGORY;
SELECT TOP 10 * FROM DIM_PRODUCT;
SELECT TOP 10 * FROM DIM_SEGMENT;
SELECT TOP 10 * FROM DIM_CALENDAR;
SELECT TOP 10 * FROM FACT_SALES;

Verificar valores nulos en columnas clave
SELECT COUNT(*) AS Nulos_ID_CATEGORY FROM DIM_CATEGORY WHERE ID_CATEGORY IS NULL;
SELECT COUNT(*) AS Nulos_ITEM_CODE FROM FACT_SALES WHERE ITEM_CODE IS NULL;
SELECT COUNT(*) AS Nulos_DATE FROM DIM_CALENDAR WHERE DATE IS NULL;

Productos más vendidos por valor total
SELECT TOP 10 
    ITEM_CODE, 
    SUM(TOTAL_VALUE_SALES) AS Ventas_Totales 
FROM FACT_SALES
GROUP BY ITEM_CODE
ORDER BY Ventas_Totales DESC;

Ventas promedio por región
SELECT 
    REGION, 
    AVG(TOTAL_VALUE_SALES) AS Promedio_Ventas 
FROM FACT_SALES
GROUP BY REGION
ORDER BY Promedio_Ventas DESC;

Análisis de ventas por mes

SELECT 
    DC.MONTH, 
    SUM(FS.TOTAL_VALUE_SALES) AS Ventas_Totales 
FROM FACT_SALES FS
INNER JOIN DIM_CALENDAR DC ON FS.WEEK = DC.WEEK
GROUP BY DC.MONTH
ORDER BY DC.MONTH;

Ventas totales por categoría

SELECT 
    DC.CATEGORY, 
    SUM(FS.TOTAL_VALUE_SALES) AS Ventas_Totales
FROM FACT_SALES FS
INNER JOIN DIM_PRODUCT DP ON FS.ITEM_CODE = DP.ITEM
INNER JOIN DIM_CATEGORY DC ON DP.CATEGORY = DC.ID_CATEGORY
GROUP BY DC.CATEGORY
ORDER BY Ventas_Totales DESC;

Análisis temporal de ventas por región

SELECT 
    DC.YEAR, 
    DC.MONTH, 
    FS.REGION, 
    SUM(FS.TOTAL_VALUE_SALES) AS Ventas_Totales
FROM FACT_SALES FS
INNER JOIN DIM_CALENDAR DC ON FS.WEEK = DC.WEEK
GROUP BY DC.YEAR, DC.MONTH, FS.REGION
ORDER BY DC.YEAR, DC.MONTH, FS.REGION;



'''